In [1]:
import numpy as np
import torch
from transformers import *
from sklearn.cluster import KMeans
import time
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

You are using torchaudio==0.9.0, but torchaudio>=0.10.0 is required to use MCTCTFeatureExtractor. This requires torch>=1.10.0. Please upgrade torch and torchaudio.
You are using torch==1.9.0+cu111, but torch>=1.10.0 is required to use ViltModel. Please upgrade torch.


cuda


In [ ]:
LABELS = ['it', 'koran', 'law', 'medical', 'subtitles']
train_embeddings = {}
test_embeddings = {}
train_embeddings['sents'] = np.load('train/sents.npy')
train_embeddings['vectors'] = np.load('train/vectors.npy')
train_embeddings['labels'] = np.load('train/labels.npy')
test_embeddings['sents'] = np.load('test/sents.npy')
test_embeddings['vectors'] = np.load('test/vectors.npy')
test_embeddings['labels'] = np.load('test/labels.npy')

In [ ]:
def kmeans(train_data, num_clusters=10):
    model = KMeans(n_clusters=num_clusters,
                   max_iter=250,
                   random_state=1,
                   verbose=1)
    
    model.fit(train_data)
    return model


def get_predictions(model, test_data):
    return model.predict(test_data)


def assign_labels_to_clusters(true_labels, predictions, num_clusters):
    cluster_assignment = {}
    true_count = 0
    total = 0
    for k in num_clusters:
        count = [0] * 5
        for i, pred in enumerate(predictions):
            if pred == k:
                count[LABELS.index(true_labels[i])] += 1
        max_count = max(count)
        total_count = sum(count)
        cluster_assignment[k] = (LABELS[count.index(max_count)], max_count, total_count)
        true_count += max_count
        total += total_count
    acc = true_count / total_count
    return cluster_assignment, acc

In [ ]:
start = time.time()
model = kmeans(train_embeddings['vectors'], 10)
end  = time.time()
print(f"Time taken to converge: {(end-start) / 3600} hrs.")
pred = get_predictions(model, test_embeddings['vectors'])
cluster_assignment, accuracy = assign_labels_to_clusters(test_embeddings['labels'], pred, 10)
print(f"Overall accuracy is {accuracy:.4f}")